In [1]:
import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2
#from config import db_password

# Expand cols
pd.set_option('display.max_columns', 80)

CREATE DATASET

In [2]:
# Connction to AWS nba database
connection = psycopg2.connect(
    host = 'nba-db.cpmpsi1pfaz0.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'caf3rac3',
    database='postgres'
    )
cursor=connection.cursor()

In [3]:
#read in data from database for All NBA table
nba_unpivoted_sql = 'SELECT * FROM public.allnba_unpivoted'
All_NBA_Unpivoted_df = pd.read_sql(nba_unpivoted_sql, con=connection).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
All_NBA_Unpivoted_df.head(15)

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019


In [4]:

#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
All_NBA_Unpivoted_df.head(10)

,Season_Start_Year,PlayerName_Formatted,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2019,Anthony Davis,1,0,0,1,0,0
1,2019,Giannis Antetokounmpo,0,1,0,1,0,0
2,2019,LeBron James,0,1,0,1,0,0
3,2019,James Harden,0,0,1,1,0,0
4,2019,Luka Doncic,0,0,1,1,0,0
5,2019,Nikola Jokic,1,0,0,0,1,0
6,2019,Kawhi Leonard,0,1,0,0,1,0
7,2019,Pascal Siakam,0,1,0,0,1,0
8,2019,Damian Lillard,0,0,1,0,1,0
9,2019,Chris Paul,0,0,1,0,1,0


In [5]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
AllNBAPlayersList

array(['Anthony Davis', 'Giannis Antetokounmpo', 'LeBron James',
       'James Harden', 'Luka Doncic', 'Nikola Jokic', 'Kawhi Leonard',
       'Pascal Siakam', 'Damian Lillard', 'Chris Paul', 'Rudy Gobert',
       'Jayson Tatum', 'Jimmy Butler', 'Ben Simmons', 'Russell Westbrook',
       'Paul George', 'Stephen Curry', 'Joel Embiid', 'Kevin Durant',
       'Kyrie Irving', 'Blake Griffin', 'Kemba Walker',
       'LaMarcus Aldridge', 'DeMar DeRozan', 'Karl-Anthony Towns',
       'Victor Oladipo', 'Isaiah Thomas', 'DeAndre Jordan',
       'Draymond Green', 'John Wall', 'DeMarcus Cousins',
       'Andre Drummond', 'Klay Thompson', 'Kyle Lowry', 'Marc Gasol',
       'Pau Gasol', 'Tim Duncan', 'Joakim Noah', 'Dwight Howard',
       'Kevin Love', 'Tony Parker', 'Al Jefferson', 'Goran Dragic',
       'Kobe Bryant', 'Carmelo Anthony', 'David Lee', 'Dwyane Wade',
       'Andrew Bynum', 'Tyson Chandler', 'Dirk Nowitzki', 'Rajon Rondo',
       'Derrick Rose', "Amar'e Stoudemire", 'Al Horford', 'Za

In [6]:
#read in data from database for season stats table
seasons_stats_sql = 'SELECT * FROM public.seasons_stats'
season_stats_df = pd.read_sql(seasons_stats_sql, con=connection)
#divid year into season start and end
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_end'] = pd.to_numeric(season_stats_df['Year_season_end'])
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df.head()


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_season_start
0,1980,Tim Bassett,PF,28,TOT,12,0,164.0,6.7,0.419,0.000,0.441,7.1,14.0,10.6,9.7,2.2,0.0,18.1,11.6,-0.1,0.2,0.1,0.033,-3.8,0.3,-3.6,-0.1,12.0,34.0,0.353,0.0,0.0,NaN,12.0,34.0,0.353,0.353,10.0,15.0,0.667,11.0,22.0,33.0,14.0,8.0,0.0,9.0,27.0,34.0,1979
1,1980,Tim Bassett,PF,28,NJN,7,0,92.0,7.9,0.440,0.000,0.545,7.9,12.4,10.1,5.5,2.5,0.0,12.8,13.1,0.0,0.1,0.1,0.063,-3.7,0.1,-3.5,0.0,8.0,22.0,0.364,0.0,0.0,NaN,8.0,22.0,0.364,0.364,8.0,12.0,0.667,7.0,11.0,18.0,4.0,5.0,0.0,4.0,14.0,24.0,1979
2,1980,M.L. Carr,SF,29,BOS,82,7,1994.0,14.6,0.526,0.054,0.316,6.0,12.3,9.2,10.7,2.8,1.0,14.1,20.2,1.9,3.2,5.2,0.125,-0.2,1.5,1.3,1.7,362.0,763.0,0.474,12.0,41.0,0.293,350.0,722.0,0.485,0.482,178.0,241.0,0.739,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0,1979
3,1980,Don Chaney,SG,33,BOS,60,0,523.0,7.8,0.402,0.032,0.222,6.7,8.8,7.8,9.3,2.8,1.2,13.7,18.3,-0.4,0.8,0.4,0.033,-4.0,0.9,-3.1,-0.1,67.0,189.0,0.354,1.0,6.0,0.167,66.0,183.0,0.361,0.357,32.0,42.0,0.762,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0,1979
4,1980,Phil Chenier,SG,29,IND,23,0,380.0,8.9,0.423,0.044,0.193,2.4,7.3,4.8,15.9,1.8,1.4,15.1,17.6,-0.3,0.2,0.0,0.000,-4.3,-1.6,-5.8,-0.4,52.0,135.0,0.385,2.0,6.0,0.333,50.0,129.0,0.388,0.393,18.0,26.0,0.692,9.0,26.0,35.0,47.0,15.0,10.0,26.0,29.0,124.0,1979


In [7]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [8]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [9]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [10]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))

In [11]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()
Positions_List

array(['PF', 'SF', 'SG', 'C', 'PG', 'SF-SG', 'SG-SF', 'C-PF', 'PF-C',
       'SF-PF', 'PG-SG', 'SG-PG', 'PF-SF', 'PG-SF', 'SG-PF', 'C-SF'],
      dtype=object)

In [12]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [13]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )

In [14]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)

In [15]:
# drop year  and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_start'],axis=1)

Season_Stats_ML_DF.columns

Index(['Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS_pct', '3PAr', 'FTr', 'ORB_pct',
       'DRB_pct', 'TRB_pct', 'AST_pct', 'STL_pct', 'BLK_pct', 'TOV_pct',
       'USG_pct', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG_pct', '3P', '3PA', '3P_pct', '2P', '2PA', '2P_pct',
       'eFG_pct', 'FT', 'FTA', 'FT_pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'AllNBA_Center', 'AllNBA_Forward',
       'AllNBA_Guard', 'Tm_1st', 'Tm_2nd', 'Tm_3rd', 'is_Guard', 'is_Forward',
       'is_Center'],
      dtype='object')

In [16]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [17]:
Season_Stats_ML_DF['3PAr'] = Season_Stats_ML_DF['3PAr'].fillna(int(0))
Season_Stats_ML_DF['3P'] = Season_Stats_ML_DF['3P'].fillna(int(0))
Season_Stats_ML_DF['3PA'] = Season_Stats_ML_DF['3PA'].fillna(int(0))
Season_Stats_ML_DF['3P_pct'] = Season_Stats_ML_DF['3P_pct'].fillna(int(0))


In [18]:
#make dummies for team
#Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm'])
#I don't like this lol

In [19]:
# Fill null values with zero
for col in list(Season_Stats_ML_DF.columns):
    Season_Stats_ML_DF[col] = Season_Stats_ML_DF[col].fillna(int(0))

ALL NBA CENTER MACHINE LEARNING SECTION

In [20]:
#Create Dataset for X & y, make sure it is only centers...we don't need guard or forward data her

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
y = y["AllNBA_Center"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_Center_Train = X_train

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

center_model = model

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix Centers")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix Centers


,Predicted 0,Predicted 1
Actual 0,925,5
Actual 1,9,15


Accuracy Score : 0.9853249475890985
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       930
         1.0       0.75      0.62      0.68        24

    accuracy                           0.99       954
   macro avg       0.87      0.81      0.84       954
weighted avg       0.98      0.99      0.98       954



In [21]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
center_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
center_feature_importance

[(0.8087160177529339, 'WS'),
 (0.06899112254734718, 'SPG'),
 (0.049650468074336505, 'RPG'),
 (0.021370777460871532, 'DBPM'),
 (0.007346831193775636, 'PF'),
 (0.006013866169930714, 'TOV'),
 (0.004850845976900046, 'Age'),
 (0.0041927620658712495, 'USG_pct'),
 (0.004109352525386954, 'PER'),
 (0.004080856480164123, 'FT'),
 (0.003510242589311578, 'OBPM'),
 (0.003486835643464389, 'AST'),
 (0.0021473519729513784, 'DRB_pct'),
 (0.0020857761548553353, '3P_pct'),
 (0.0015867596695141317, 'FG_pct'),
 (0.0014048235453448248, '2P'),
 (0.0014024394578687142, 'FTr'),
 (0.0007171762589928051, 'TOV_pct'),
 (0.0007127078710863027, 'BLK_pct'),
 (0.0007116679584421123, 'ORB_pct'),
 (0.0007109994117456671, '3P'),
 (0.0007094324540367715, 'PTS'),
 (0.000708325566673182, 'TRB_pct'),
 (0.0007045050801535998, 'DWS'),
 (6.940005534559504e-05, 'PPG'),
 (8.656062695542293e-06, '2P_pct'),
 (0.0, 'eFG_pct'),
 (0.0, 'VORP'),
 (0.0, 'TS_pct'),
 (0.0, 'TRB'),
 (0.0, 'STL'),
 (0.0, 'OWS'),
 (0.0, 'ORB'),
 (0.0, 'MP'),


ALL NBA GUARD MACHINE LEARNING SECTION

In [22]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
y = y["AllNBA_Guard"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_Guard_Train = X_train
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier(random_state=78)

model = model.fit(X_train_scaled, y_train)

guard_model = model

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1809,20
Actual 1,25,35


Accuracy Score : 0.9761778718898888
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1829
         1.0       0.64      0.58      0.61        60

    accuracy                           0.98      1889
   macro avg       0.81      0.79      0.80      1889
weighted avg       0.98      0.98      0.98      1889



In [23]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
guard_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
guard_feature_importance

[(0.7759533681106554, 'WS'),
 (0.07399570248740046, 'PER'),
 (0.04102197655661936, 'FT'),
 (0.029637970265626676, 'DBPM'),
 (0.007705705372945374, '3PA'),
 (0.007006371231496459, 'MP'),
 (0.006150510106706786, 'VORP'),
 (0.0047615579700718, 'FTr'),
 (0.004271953092279772, '2P_pct'),
 (0.0039324494087095855, 'ORB_pct'),
 (0.0038923676271178883, 'TRB_pct'),
 (0.0035247985214068448, 'Age'),
 (0.0030148583276531893, 'DWS'),
 (0.0027987072226163144, 'STL'),
 (0.0027460300244987407, 'FTA'),
 (0.0026608805127680208, 'AST_pct'),
 (0.0021914198630122866, 'FG'),
 (0.002081039910081163, 'TOV'),
 (0.0020515391452199994, 'BPM'),
 (0.0020276613054634207, '2P'),
 (0.0017603790577955555, '3P_pct'),
 (0.001698368499342982, 'USG_pct'),
 (0.0016955756196098395, 'TS_pct'),
 (0.0014083543743059108, 'OWS'),
 (0.0014068637128322396, '3PAr'),
 (0.0013881172239891405, 'G'),
 (0.001091770326764453, 'BLK'),
 (0.0010741403741329565, 'RPG'),
 (0.0010357271194528762, '3P'),
 (0.0007159721457782632, 'DRB_pct'),
 (0.

ALL NBA FORWARD MACHINE LEARNING SECTION

In [24]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
y = y["AllNBA_Forward"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_Forward_Train = X_train
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)
forward_model = model.fit(X_train_scaled, y_train)
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix Forwards")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix Forwards


,Predicted 0,Predicted 1
Actual 0,1851,21
Actual 1,20,25


Accuracy Score : 0.9786124152321335
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1872
         1.0       0.54      0.56      0.55        45

    accuracy                           0.98      1917
   macro avg       0.77      0.77      0.77      1917
weighted avg       0.98      0.98      0.98      1917



In [25]:
sum(predictions)

46.0

In [26]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
forward_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
forward_feature_importance

[(0.8267637033994606, 'PPG'),
 (0.03940688812688925, 'VORP'),
 (0.03047834298504556, 'DRB'),
 (0.01841134473907527, 'BPM'),
 (0.013559055165375434, 'PER'),
 (0.012663585802712951, 'USG_pct'),
 (0.008153134001433734, 'WS'),
 (0.005448957797213942, 'DWS'),
 (0.005033727174886673, '3P'),
 (0.0044308677668722036, 'AST_pct'),
 (0.003502689786152544, 'FTr'),
 (0.00346386737588827, 'BLK'),
 (0.003360965956801836, 'TOV_pct'),
 (0.0033133909906491493, '3PA'),
 (0.002418826466684103, 'TOV'),
 (0.0023552316212564744, 'BLK_pct'),
 (0.0017416646767498933, 'DBPM'),
 (0.001738154451508075, '3PAr'),
 (0.0017312748227399208, 'FT'),
 (0.0016963615638889912, 'OWS'),
 (0.0016075876416127767, '2P_pct'),
 (0.001379099288900026, 'ORB'),
 (0.0010621544939619933, 'TRB'),
 (0.001019440873036668, 'SPG'),
 (0.000711432210341434, 'TRB_pct'),
 (0.0007001753760686476, '2P'),
 (0.0006933098721882791, '3P_pct'),
 (0.0006578856870152206, 'ORB_pct'),
 (0.0003611064088049672, 'AST'),
 (0.0003507236542502591, 'Age'),
 (0.

Making Predictions

In [27]:
filename =  r'C:\Users\jack.getman\OneDrive - WaterBridge Resources, LLC\Desktop\DatacampNew\Bootcamp_Final_Project\2021Data\season_stats2021.csv'
seasons_stats_2021 = pd.read_csv(filename) 
try:
    seasons_stats_2021 = seasons_stats_2021.drop(['Unnamed: 0'], axis=1)
except:
    print('switch the data source')

#Creating Per Game Statistics

#Points per Game
seasons_stats_2021["PPG"] = seasons_stats_2021["PTS"]/seasons_stats_2021["G"]
#Assits Per Game
seasons_stats_2021["APG"] = seasons_stats_2021["AST"]/seasons_stats_2021["G"]
#Rebounds Per Game
seasons_stats_2021["RPG"] = seasons_stats_2021["TRB"]/seasons_stats_2021["G"]
#Blocks Per Game
seasons_stats_2021["RPG"] = seasons_stats_2021["BLK"]/seasons_stats_2021["G"]
#STLs Per Game
seasons_stats_2021["SPG"] = seasons_stats_2021["STL"]/seasons_stats_2021["G"]    
    




In [28]:
filename =  r'C:\Users\jack.getman\OneDrive - WaterBridge Resources, LLC\Desktop\DatacampNew\Bootcamp_Final_Project\Stat_Glossary.xlsx'
stat_glossary = pd.read_excel(filename) 
    
    
scaled_stats = stat_glossary[stat_glossary.Scaled_Per_Game != 0].reset_index().drop(['index'], axis=1).Stat

scaled_seasons_stats_2021 = seasons_stats_2021

def scale_stats(season_df, num_games):
    #scaled_season_df = pd.DataFrame()
    for stat in scaled_stats:
        season_df[stat] = season_df[stat] * 82/num_games
    return season_df

#seasons_stats_2021[seasons_stats_2021.Player == 'LeBron James']

In [29]:
#seasons_stats_2021 = scale_stats(seasons_stats_2021,72)
#seasons_stats_2021.head()

In [30]:
seasons_stats_2021[seasons_stats_2021.Player == 'Stephen Curry']

,Player,Pos,Age,Tm,G,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GS,PPG,APG,RPG,SPG
115,Stephen Curry,PG,32,GSW,63,2152,26.3,0.655,0.587,0.289,1.5,15.4,8.5,30.5,1.7,0.3,12.2,34.8,6.5,2.5,9.0,0.201,8.1,0.0,8.1,5.5,658,1365,0.482,337,801,0.421,321,564,0.569,0.605,362,395,0.916,29,316,345,363,77,8,213,119,2015,63,31.984127,5.761905,0.126984,1.222222


In [31]:
centers_stats_2021 = seasons_stats_2021
forwards_stats_2021 = seasons_stats_2021
guards_stats_2021 = seasons_stats_2021

#seasons_stats_2021=seasons_stats_2021[['Age','G','GS','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','STL_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','WS/48','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']]
#seasons_stats_2021

centers_stats_2021['is_Center'] = centers_stats_2021['Pos'].apply(lambda pos: is_center(pos) )
guards_stats_2021['is_Guard'] = guards_stats_2021['Pos'].apply(lambda pos: is_guard(pos) )
forwards_stats_2021['is_Forward'] = forwards_stats_2021['Pos'].apply(lambda pos: is_forward(pos) )

centers_stats_2021=centers_stats_2021[centers_stats_2021['is_Center']==1]
guards_stats_2021=guards_stats_2021[guards_stats_2021['is_Guard']==1]
forwards_stats_2021=forwards_stats_2021[forwards_stats_2021['is_Forward']==1]

centers_stats_2021=centers_stats_2021[centers_stats_2021['is_Center']==1].reset_index()
guards_stats_2021=guards_stats_2021[guards_stats_2021['is_Guard']==1].reset_index()
forwards_stats_2021=forwards_stats_2021[forwards_stats_2021['is_Forward']==1].reset_index()

#centers_stats_2021=centers_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']]
#guards_stats_2021=guards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']]
#forwards_stats_2021=forwards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']]

In [32]:
#Predict All- nba Centers
X_scaler = StandardScaler().fit(X_Center_Train)
X_test_scaled = X_scaler.transform(centers_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct',
                                                       'DRB_pct','TRB_pct','AST_pct','BLK_pct',
                                                       'TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM',
                                                       'DBPM','BPM','VORP','FG','FGA','FG_pct','3P',
                                                       '3PA','3P_pct','2P','2PA','2P_pct','eFG_pct',
                                                       'FT','FTA','FT_pct','ORB','DRB','TRB','AST',
                                                       'STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']])
predictions = center_model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns = {predictions_df.columns[0]:'All_NBA_Status'})
centers_stats_2021=centers_stats_2021.join(predictions_df)
centers_stats_2021[centers_stats_2021.All_NBA_Status == 1]

,index,Player,Pos,Age,Tm,G,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward,All_NBA_Status
28,145,Joel Embiid,C,26,PHI,51,1585,30.3,0.636,0.171,0.610,8.0,29.1,18.7,16.2,1.5,3.9,12.2,35.3,5.6,3.2,8.8,0.266,6.2,1.0,7.2,3.7,461,899,0.513,58,154,0.377,403,745,0.541,0.545,471,548,0.859,113,426,539,145,50,69,159,123,1451,51,28.450980,2.843137,1.352941,0.980392,1,0,0,1.0
37,178,Rudy Gobert,C,28,UTA,71,2187,23.5,0.683,0.007,0.646,12.2,33.5,23.3,6.0,0.9,7.0,13.7,17.0,6.1,5.2,11.3,0.248,2.0,2.5,4.5,3.6,391,579,0.675,0,4,0.000,391,575,0.680,0.675,233,374,0.623,240,720,960,89,40,190,118,163,1015,71,14.295775,1.253521,2.676056,0.563380,1,0,0,1.0


In [33]:
#Predict All- nba Guards
X_scaler = StandardScaler().fit(X_Guard_Train)
X_test_scaled = X_scaler.transform(guards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct',
                                                      'DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct',
                                                      'OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA',
                                                      'FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct',
                                                      'FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK',
                                                      'TOV','PF','PTS','PPG','RPG','SPG']])
predictions = guard_model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns = {predictions_df.columns[0]:'All_NBA_Status'})
guards_stats_2021=guards_stats_2021.join(predictions_df)
guards_stats_2021[guards_stats_2021.All_NBA_Status == 1]

,index,Player,Pos,Age,Tm,G,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward,All_NBA_Status
226,516,Russell Westbrook,PG,32,WAS,65,2369,19.5,0.509,0.221,0.334,4.9,28.6,16.8,48.6,1.7,0.8,18.0,30.2,0.5,3.2,3.7,0.075,2.5,0.8,3.4,3.2,544,1238,0.439,86,273,0.315,458,965,0.475,0.474,271,413,0.656,109,641,750,763,89,23,312,190,1445,65,22.230769,11.738462,0.353846,1.369231,0,1,0,1.0
232,537,Trae Young,PG,22,ATL,63,2125,23.0,0.589,0.357,0.491,2.0,10.5,6.3,45.5,1.2,0.5,16.2,33.0,5.9,1.3,7.2,0.163,5.4,-1.7,3.6,3.0,487,1112,0.438,136,397,0.343,351,715,0.491,0.499,484,546,0.886,38,207,245,594,53,12,261,111,1594,63,25.301587,9.428571,0.190476,0.841270,0,1,0,1.0


In [34]:
#Predict All- nba Forwards
X_scaler = StandardScaler().fit(X_Forward_Train)
X_test_scaled = X_scaler.transform(forwards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct',
                                                        'TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS',
                                                        'WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA',
                                                        '3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB',
                                                        'DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']])
predictions = forward_model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns = {predictions_df.columns[0]:'All_NBA_Status'})
forwards_stats_2021=forwards_stats_2021.join(predictions_df)
forwards_stats_2021[forwards_stats_2021.All_NBA_Status == 1]

,index,Player,Pos,Age,Tm,G,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward,All_NBA_Status
3,11,Giannis Antetokounmpo,PF,26,MIL,61,2013,29.2,0.633,0.201,0.528,5.3,28.9,17.5,28.7,1.7,3.2,13.2,32.5,6.9,3.3,10.2,0.244,6.2,2.6,8.8,5.5,626,1100,0.569,67,221,0.303,559,879,0.636,0.6,398,581,0.685,97,574,671,357,72,73,207,168,1717,61,28.147541,5.852459,1.196721,1.180328,0,0,1,1.0


In [35]:
# Make sure that the games played for 2021 is normalized to 82 games
# 72 games in 2021 
# 1998-1999 50 games
# 2011-2021 66 games